In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc2


In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:1.264387 auc:0.5031
epoch:  20 loss:0.245374 auc:0.9523
epoch:  40 loss:0.184093 auc:0.9565
epoch:  60 loss:0.161142 auc:0.9599
epoch:  80 loss:0.151510 auc:0.9639
epoch: 100 loss:0.142864 auc:0.9677
epoch: 120 loss:0.137201 auc:0.9696
epoch: 140 loss:0.134810 auc:0.9701
epoch: 160 loss:0.133132 auc:0.9705
epoch: 180 loss:0.131789 auc:0.9708
epoch: 200 loss:0.130553 auc:0.9711
epoch: 220 loss:0.129442 auc:0.9715
epoch: 240 loss:0.128657 auc:0.9719
epoch: 260 loss:0.126855 auc:0.9725
epoch: 280 loss:0.125776 auc:0.9730
epoch: 300 loss:0.123791 auc:0.9735
epoch: 320 loss:0.123336 auc:0.9739
epoch: 340 loss:0.121152 auc:0.9738
epoch: 360 loss:0.119997 auc:0.9749
epoch: 380 loss:0.120766 auc:0.9750
epoch: 400 loss:0.117307 auc:0.9749
epoch: 420 loss:0.115351 auc:0.9760
epoch: 440 loss:0.114678 auc:0.9763
epoch: 460 loss:0.119338 auc:0.9747
epoch: 480 loss:0.114933 auc:0.9767
epoch: 500 loss:0.111943 auc:0.9766
epoch: 520 loss:0.110969 auc:0.9771
epoch: 540 loss:0.109833 auc

In [6]:
true_datas.reset_index(drop=True).to_csv("true_gdsc2.csv")
predict_datas.reset_index(drop=True).to_csv("pred_gdsc2.csv")